In [1]:
!pip install gradio joblib pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the dataset
dataset_path = "LoanApprovalPrediction.csv"
loan_data = pd.read_csv(dataset_path)

# Check for missing values
print("Missing values in each column:")
print(loan_data.isnull().sum())

# Fill missing values or drop rows
loan_data = loan_data.dropna()

# Display the dataset info after handling missing values
loan_data.info()

# Identify categorical features (excluding Loan_ID and Loan_Status)
categorical_features = loan_data.select_dtypes(include=['object']).columns.drop(['Loan_ID','Loan_Status'])

# Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', loan_data.select_dtypes(exclude=['object']).columns),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)])

# Apply preprocessing to the features
X = preprocessor.fit_transform(loan_data.drop(columns=['Loan_Status']))

# Encode the target variable using Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(loan_data['Loan_Status'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Check accuracy on the training set
train_accuracy = model.score(X_train, y_train)
print(f"Training accuracy: {train_accuracy * 100:.2f}%")

Missing values in each column:
Loan_ID               0
Gender                0
Married               0
Dependents           12
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           21
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 1 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            505 non-null    object 
 1   Gender             505 non-null    object 
 2   Married            505 non-null    object 
 3   Dependents         505 non-null    float64
 4   Education          505 non-null    object 
 5   Self_Employed      505 non-null    object 
 6   ApplicantIncome    505 non-null    int64  
 7   CoapplicantIncome  505 non-null    float64
 8   LoanAmount         505 non-null    float64
 9   Loan_Amount

In [5]:
from sklearn.metrics import classification_report, accuracy_score

# Make predictions on the test data
y_pred = model.predict(X_test)

# Display classification report and accuracy
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy on test data:", accuracy_score(y_test, y_pred) * 100, "%")


Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.64      0.70        25
           1       0.89      0.93      0.91        76

    accuracy                           0.86       101
   macro avg       0.82      0.79      0.80       101
weighted avg       0.86      0.86      0.86       101

Accuracy on test data: 86.13861386138613 %


In [6]:
!pip install --upgrade gradio


In [9]:
import gradio as gr
import joblib
import pandas as pd

# Load the model (ensure it's uploaded or available in the directory)
joblib.dump(model, "home_loan_model.pkl")
model = joblib.load("home_loan_model.pkl")

# Preprocessing function for encoding categorical features
def preprocess_input(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area):
    gender_map = {"Male": 0, "Female": 1}
    married_map = {"Married": 1, "Not Married": 0}
    education_map = {"Graduate": 1, "Not Graduate": 0}
    self_employed_map = {"Self-employed": 1, "Not Self-employed": 0}
    property_area_map = {"Urban": 2, "Semiurban": 1, "Rural": 0}

    dependents = 0 if dependents == "" else int(dependents)
    loan_amount = 0 if loan_amount == "" else float(loan_amount)
    loan_amount_term = 0 if loan_amount_term == "" else int(loan_amount_term)
    credit_history = 0 if credit_history == "" else float(credit_history)

    input_data = pd.DataFrame({
        "Gender": [gender_map.get(gender, 0)],
        "Married": [married_map.get(married, 0)],
        "Dependents": [dependents],
        "Education": [education_map.get(education, 0)],
        "Self_Employed": [self_employed_map.get(self_employed, 0)],
        "ApplicantIncome": [float(applicant_income)],
        "CoapplicantIncome": [float(coapplicant_income)],
        "LoanAmount": [float(loan_amount)],
        "Loan_Amount_Term": [loan_amount_term],
        "Credit_History": [credit_history],
        "Property_Area": [property_area_map.get(property_area, 0)]
    })

    return input_data

# Prediction function
def predict_home_loan(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area):
    # Preprocess the input data
    input_data = preprocess_input(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area)

    # Make prediction using the model
    prediction = model.predict(input_data)
    result = "Approved" if prediction[0] == 1 else "Rejected"

    # Return the result
    return result

# Create the Gradio interface with custom layout and styling
css = """
    .gradio-container {
        background-color: #f9f9f9;  # Light background
        font-family: 'Arial', sans-serif;
        padding: 20px;
        border-radius: 10px;
    }
    .gradio-input {
        font-size: 16px;
        padding: 12px;
    }
    .gradio-button {
        background-color: #4CAF50;
        color: white;
        font-size: 16px;
        padding: 12px;
        width: 100%;
    }
    .gradio-output {
        font-size: 18px;
        font-weight: bold;
        padding: 10px;
    }
"""

# Create Gradio interface using Blocks for better layout control
with gr.Blocks(css=css) as interface:
    with gr.Row():
        with gr.Column():
            gr.Dropdown(choices=["Male", "Female"], label="Gender")
            gr.Dropdown(choices=["Married", "Not Married"], label="Married")
            gr.Number(label="Dependents")
            gr.Dropdown(choices=["Graduate", "Not Graduate"], label="Education")
            gr.Dropdown(choices=["Self-employed", "Not Self-employed"], label="Self Employed")
            gr.Number(label="Applicant Income")
            gr.Number(label="Coapplicant Income")
            gr.Number(label="Loan Amount")
            gr.Number(label="Loan Term (months)")
            gr.Number(label="Credit History (0 or 1)")
            gr.Dropdown(choices=["Urban", "Semiurban", "Rural"], label="Property Area")
        with gr.Column():
            gr.Textbox(label="Loan Approval Status", interactive=False, lines=2)

    # Launch the Gradio interface
    interface.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>